<a href="https://colab.research.google.com/github/hamidkhadem/face-detector-p22089/blob/main/cascade_IUO_metric.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## mount google drive and download data set

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Download wiki dataset from this link:https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/
!wget https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/wiki.tar.gz -P "/content/drive/MyDrive/Colab Notebooks/"

# extract the zip file - wiki.tar.gz
!tar -xvf "/content/drive/MyDrive/Colab Notebooks/wiki.tar.gz" -C "/content/drive/MyDrive/Colab Notebooks/"


Streaming output truncated to the last 5000 lines.
wiki/50/8653450_1985-11-28_2014.jpg
wiki/50/28040650_1986-04-20_2011.jpg
wiki/50/1896450_1928-03-22_1990.jpg
wiki/50/39477450_1965-01-04_2014.jpg
wiki/50/17106250_1950-02-23_2010.jpg
wiki/50/17508150_1986-04-22_2012.jpg
wiki/50/15701850_1986-04-15_2010.jpg
wiki/50/28037450_1934-07-09_2002.jpg
wiki/50/14040750_1989-10-04_2008.jpg
wiki/50/28991050_1964-08-19_2010.jpg
wiki/50/27962750_1894-03-20_1946.jpg
wiki/50/29127850_1923-11-05_2007.jpg
wiki/50/33309450_1989-04-27_2012.jpg
wiki/50/20291650_1987-02-18_2014.jpg
wiki/50/2529850_1922-12-05_1966.jpg
wiki/50/42305150_1954-07-11_1950.jpg
wiki/50/22659150_1890-11-01_1962.jpg
wiki/50/29569950_1978-04-28_1999.jpg
wiki/50/7720450_1956-07-18_2010.jpg
wiki/50/7973150_1985-10-07_2009.jpg
wiki/50/2744250_1981-01-17_2010.jpg
wiki/50/166450_1927-06-15_1989.jpg
wiki/50/35796450_1936-01-21_1973.jpg
wiki/50/43761250_1941-04-20_1960.jpg
wiki/50/38389550_1970-03-25_1996.jpg
wiki/50/683150_1986-06-27_2007.j

## Reading dataset's mat file:
We are using pymatreader module to read mat file.

In [ ]:
# install pymatreader package
!pip install pymatreader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Finding** face box of in image by reading wiki.mat file and pymatreader library.

In [ ]:
# using pymatreader lib to read mat file
from pymatreader import read_mat

# path of mat file
path_mat_file = "/content/drive/MyDrive/Colab Notebooks/wiki/wiki.mat"
# load wiki.mat on a variable
mat = read_mat(path_mat_file)

In [ ]:
# print info in mat dictionary
print(mat.keys())
print(mat['__header__'])
print(mat['__version__'])
print(mat['__globals__'])
print(mat['wiki'].keys())
print(mat['wiki']['gender'][133])

dict_keys(['__header__', '__version__', '__globals__', 'wiki'])
b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sat Jan 16 16:25:20 2016'
1.0
[]
dict_keys(['dob', 'photo_taken', 'full_path', 'gender', 'name', 'face_location', 'face_score', 'second_face_score'])
1.0


In [ ]:
print(f"type of mat['wiki']['full_path'] is {type(mat['wiki']['full_path'])}")
# print total number of images in wiki dataset
print(f"lenth of mat['wiki']['full_path'] list is : {len(mat['wiki']['full_path'])}")
# print location of one image in wiki dataset
print(mat['wiki']['face_location'][6])

type of mat['wiki']['full_path'] is <class 'list'>
lenth of mat['wiki']['full_path'] list is : 62328
[ 79.35580189  26.65993396 197.60950472 144.91363679]


## Intersection over Union (IOU) accuracy score:
defined a function that calculate of IOU from to boxes, one from wiki mat file another from haar cascade detection.


In [ ]:
# Example a box input is: { 'width': 156, 'top': 61, 'height': 142, 'left': 363 }
def intersection_over_union(box_a, box_b):
    # Determine the coordinates of each of the two boxes
    xA = max(box_a['left'], box_b['left'])
    yA = max(box_a['top'], box_b['top'])
    xB = min(box_a['left']+box_a['width'], box_b['left']+box_b['width'])
    yB = min(box_a['top']+box_a['height'], box_b['top']+box_b['height'])
      
    # Calculate the area of the intersection area
    area_of_intersection = (xB - xA + 1) * (yB - yA + 1)
  
    # Calculate the area of both rectangles
    box_a_area = (box_a['width'] + 1) * (box_a['height'] + 1)
    box_b_area = (box_b['width'] + 1) * (box_b['height'] + 1)

    # Calculate the area of intersection divided by the area of union
    # Area of union = sum both areas less the area of intersection
    iou = area_of_intersection / float(box_a_area + box_b_area - area_of_intersection)
    
    # Return the score
    return iou

## haar cascade face detection
First defining the face detection function which get a cascade, number of image in wiki dataset and mat file of wiki dataset. at the end return the dictionary of face box to use for calculate the IOU.

In [ ]:
import cv2
# Face detection Function return dic of face location
# Example a = { 'width': 156, 'top': 61, 'height': 142, 'left': 363 }
def detect_faces(cascade, num_of_img, mat):

  # cause of there are several images in wiki data set that not have a face or other exception
  # we use try to skip this error in wiki dataset
  try:

    # general path to dataset
    path_image = '/content/drive/MyDrive/Colab Notebooks/wiki/'
    # read image from dataset
    img = cv2.imread(path_image + mat['wiki']['full_path'][i])
    # create a copy of the image to prevent any changes to the original one.
    image_copy = img.copy()

    #convert the test image to gray scale as opencv face detector expects gray images
    gray_image = cv2.cvtColor(image_copy, cv2.COLOR_BGR2GRAY)
    # Applying the haar classifier to detect faces
    faces_rect = cascade.detectMultiScale(gray_image)
    
    # get value of first face detection
    x, y, w, h = faces_rect[0]
    # Example box = { 'width': 156, 'top': 61, 'height': 142, 'left': 363 }
    box = { 'width': w, 'top': y, 'height': h, 'left': x }
    return box
  # if found exception return None value
  except:
    return None

## Change the format of mat face loaction

In [ ]:
# defined a function that return dictionary of mat face location with desired format:
# Example a = { 'width': 156, 'top': 61, 'height': 142, 'left': 363 }
def mat_face_box(num_of_img, mat):
    # get location of face from mat file
    box = mat['wiki']['face_location'][num_of_img]
    # change the format of box to example dictionary
    box_dic = {'width': int(box[2]), 'top': int(box[1]), 'height': int(box[3]), 'left': int(box[0])}

    return box_dic

## Calculate of avrage of all IOU images in wiki dataset

I use this command to create a haar cascade frontal face ditection object to find location of face in image.
```
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
```
after that we run a loop to calculate IOU for every valid image in dataset and sum all to calculate average IOU.


In [ ]:
import cv2

# load haar cascade face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades 
                                     + 'haarcascade_frontalface_default.xml')
# find the total number of image in wiki dataset - 62328
num_total_img = len(mat['wiki']['full_path'])

# defined value that save sum all iou of images
total_iou = 0
# cause skipping some of image we have to count image to calculate average
count_img_with_face = 0
# run a loop in desired range to calculate sum of All IOU images in range
# i is number of images in wiki dataset
for i in range(1000):
    # get the box from cascade detection
    box_cascade = detect_faces(face_cascade, i, mat)
    # check if there is no face in image skip it
    if box_cascade == None:
      continue
    else:
      # add one to valid image counter
      count_img_with_face += 1
      # reformat face location in mat file for IOU function
      box_mat = mat_face_box(i, mat)
      # calculate IOU for iamge and add to sum of all IOU variable
      total_iou += intersection_over_union(box_cascade, box_mat)

# calculate the avrage of Overall IOU in wiki dataset
avg_iou = total_iou / count_img_with_face
print(f"Average of IOU overall all images in wiki dataset is:  {avg_iou}")

Average of IOU overall all images in wiki dataset is:  0.2175108199699261
